
- Este caderno tem como objetivo eliminar eventuais variáveis independentes constantes, quase-constantes (com variância de valores muito baixa), duplicadas ou com correlação alta, e verificar, a partir do score ROC AUC, se os modelos melhoram.
- Foram geradas random forests e modelos de regressão logística. Para a regressão logística, foram eliminados todos os registros com dados faltantes e utilizado o StandardScaler.
- O ROC AUC score é uma métrica de avaliação de desempenho para modelos de classificação binária. Ele representa a área sob a curva ROC (Receiver Operating Characteristic), que plota a taxa de verdadeiros positivos (sensibilidade) versus a taxa de falsos positivos (1 - especificidade) em vários limiares de classificação.
- Um valor de AUC de 0,5 indica um modelo que não é melhor do que o acaso. Um valor de AUC de 1,0 indica um modelo perfeito. Quanto maior o valor da AUC, melhor o modelo.

Observações gerais:
- Para a random forest, dez variáveis foram consideradas constantes (COBRADE_11110', 'COBRADE_11120', 'COBRADE_11312', 'COBRADE_11313', 'COBRADE_13310', 'COBRADE_13321', 'COBRADE_14140', 'COBRADE_15120', 'COBRADE_22210', 'COBRADE_25100');
- E 26 variáveis foram consideradas quase constantes (COBRADE_11311, COBRADE_11321, COBRADE_11331, COBRADE_11332,
COBRADE_11340, COBRADE_11410, COBRADE_11420, COBRADE_11431,
COBRADE_11432, COBRADE_11433, COBRADE_13111, COBRADE_13112,  COBRADE_13120, COBRADE_13211, COBRADE_13212, COBRADE_13322, COBRADE_14132, COBRADE_15110, COBRADE_15130, COBRADE_15210, COBRADE_15230, COBRADE_22220, COBRADE_23120,COBRADE_24100, COBRADE_24200, COBRADE_25500);
- Não havia variáveis duplicadas;
- Utilizando-se 0,7 como threshold, foram identificadas duas variáveis correlacionadas (PDEFLIXO', 'PDEFSAN');
- Assim, foram gerados modelos com 22 das 60 variáveis independentes iniciais 8'DH_total_danos_humanos', 'DM_total_danos_materiais', 'PEPL_total_publico','PEPR_total_privado', 'regiao_Centro-oeste',       'regiao_Nordeste', 'regiao_Norte', 'regiao_Sudeste', 'regiao_Sul', 'COBRADE_12100', 'COBRADE_12200','COBRADE_12300', 'COBRADE_13213', 'COBRADE_13214', 'COBRADE_13215', 'COBRADE_14110', 'COBRADE_14120',        'COBRADE_14131', 'DOMICILIO_AREARURAL', 'PDEFAGUA', 'PDEFESGOTO', 'QTDE_FAMILIAS_ATUALIZADAS').

###Performance dos modelos:

**Random forest com dataset original:**
Train set roc-auc: 0.806602600300164
Test set roc-auc: 0.774244647147626

**Random forest com filtro básico (sem constantes e quase constantes):**
Train set roc-auc: 0.8149491455650816
Test set roc-auc: 0.7776588086277441

**Random forest com filtro básico + eliminando correlatas:**
Train set roc-auc: 0.81341569637481
Test set roc-auc: 0.7758063070862076

**Regressão logística com dataset original:**
Train set roc-auc: 0.7299574160481984
Test set roc-auc: 0.7296610428044055

**Regressão logística com o filtro básico (sem constantes e quase constantes):**
Train set roc-auc: 0.7234023167384853
Test set roc-auc: 0.7342412362327241

**Regressão logística com filtro básico + eliminando correlacionadas:**
Train set roc-auc: 0.721493844304521
Test set roc-auc: 0.7313550660524148


###Conclusões gerais:


- Não houve grande melhora no ROC AUC após a aplicação dos filtros. O desempenho é ligeiramente maior com a aplicação apenas dos filtros básicos.
- Os ROC AUC dos modelos com dados não agregados foram melhores que com os agregados.


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score

In [ ]:
# # load the dataset

# data = pd.read_csv('../dataset_1.csv')
# data.shape

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.float_format', '{:,.2f}'.format)
pd.set_option('display.float_format', '{:,.2f}'.format)
sns.set_theme(style="white", palette="deep", context="talk",font_scale=0.8)
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:

object_columns = ['Protocolo_S2iD', 'Nome_Municipio', 'Sigla_UF', 'regiao',
                  'Setores Censitários', 'Status', 'DH_Descricao', 'DM_Descricao',
                  'DA_Descricao', 'DA_Polui/cont da água', 'DA_Polui/cont do ar',
                  'DA_Polui/cont do solo', 'DA_Dimi/exauri hídrico',
                  "DA_Incêndi parques/APA's/APP's", 'PEPL_Descricao', 'PEPR_Descricao',
                  'Categoria', 'Grupo', 'Subgrupo', 'Tipo', 'Subtipo']

dtype = {col: 'object' for col in object_columns}

In [ ]:
df_eventos_2018_1 = pd.read_csv(
    "https://raw.githubusercontent.com/brunagmoura/PrevisorReconhecimento/refs/heads/main/df_eventos_2018_2024_modelo1.csv",
    sep=';',
    decimal=',')

#print(df_eventos_2018_1.head(5))

In [ ]:
df_eventos_2018_1.dtypes.reset_index().rename(columns={0: '.dtype', 'index': 'Variável'})

,Variável,.dtype
0,Status,int64
1,DH_total_danos_humanos,int64
2,DM_total_danos_materiais,float64
3,PEPL_total_publico,float64
4,PEPR_total_privado,float64
5,regiao_Centro-oeste,float64
6,regiao_Nordeste,float64
7,regiao_Norte,float64
8,regiao_Sudeste,float64
9,regiao_Sul,float64


In [ ]:
## Atribuir valores para X e y, sendoque y será correspondete à variável 'Status'
X, y = df_eventos_2018_1.drop('Status', axis=1), df_eventos_2018_1['Status']
#print(y)


In [ ]:
# separate dataset into train and test
# X_train, X_test, y_train, y_test = train_test_split(
#     data.drop(labels=['target'], axis=1),
#     data['target'],
#     test_size=0.3,
#     random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(X_train.shape, X_test.shape)
# X_test.head()
print(X_test.columns)

(9247, 60) (3964, 60)
Index(['DH_total_danos_humanos', 'DM_total_danos_materiais',
       'PEPL_total_publico', 'PEPR_total_privado', 'regiao_Centro-oeste',
       'regiao_Nordeste', 'regiao_Norte', 'regiao_Sudeste', 'regiao_Sul',
       'COBRADE_11110', 'COBRADE_11120', 'COBRADE_11311', 'COBRADE_11312',
       'COBRADE_11313', 'COBRADE_11321', 'COBRADE_11331', 'COBRADE_11332',
       'COBRADE_11340', 'COBRADE_11410', 'COBRADE_11420', 'COBRADE_11431',
       'COBRADE_11432', 'COBRADE_11433', 'COBRADE_12100', 'COBRADE_12200',
       'COBRADE_12300', 'COBRADE_13111', 'COBRADE_13112', 'COBRADE_13120',
       'COBRADE_13211', 'COBRADE_13212', 'COBRADE_13213', 'COBRADE_13214',
       'COBRADE_13215', 'COBRADE_13310', 'COBRADE_13321', 'COBRADE_13322',
       'COBRADE_14110', 'COBRADE_14120', 'COBRADE_14131', 'COBRADE_14132',
       'COBRADE_14140', 'COBRADE_15110', 'COBRADE_15120', 'COBRADE_15130',
       'COBRADE_15210', 'COBRADE_15230', 'COBRADE_22210', 'COBRADE_22220',
       'COBRADE_231

In [ ]:
# I keep a copy of the dataset with all the variables
# to measure the performance of machine learning models
# at the end of the notebook

X_train_original = X_train.copy()
X_test_original = X_test.copy()

X_train_no_nan = X_train.dropna()
X_test_no_nan = X_test.dropna()
y_train_no_nan = y_train[X_train_no_nan.index]
y_test_no_nan = y_test[X_test_no_nan.index]

X_train_no_nan_original = X_train_no_nan.copy()
X_test_no_nan_original = X_test_no_nan.copy()
y_train_no_nan_original = y_train_no_nan.copy()
y_test_no_nan_original = y_test_no_nan.copy()

In [ ]:
# remove constant features
constant_features = [
    feat for feat in X_train.columns if X_train[feat].std() == 0
]

print(constant_features)

X_train.drop(labels=constant_features, axis=1, inplace=True)
X_test.drop(labels=constant_features, axis=1, inplace=True)

print(X_train.shape, X_test.shape)

#X_test.head()
print(X_test.columns)

['COBRADE_11110', 'COBRADE_11120', 'COBRADE_11312', 'COBRADE_11313', 'COBRADE_13310', 'COBRADE_13321', 'COBRADE_14140', 'COBRADE_15120', 'COBRADE_22210', 'COBRADE_25100']
(9247, 50) (3964, 50)
Index(['DH_total_danos_humanos', 'DM_total_danos_materiais',
       'PEPL_total_publico', 'PEPR_total_privado', 'regiao_Centro-oeste',
       'regiao_Nordeste', 'regiao_Norte', 'regiao_Sudeste', 'regiao_Sul',
       'COBRADE_11311', 'COBRADE_11321', 'COBRADE_11331', 'COBRADE_11332',
       'COBRADE_11340', 'COBRADE_11410', 'COBRADE_11420', 'COBRADE_11431',
       'COBRADE_11432', 'COBRADE_11433', 'COBRADE_12100', 'COBRADE_12200',
       'COBRADE_12300', 'COBRADE_13111', 'COBRADE_13112', 'COBRADE_13120',
       'COBRADE_13211', 'COBRADE_13212', 'COBRADE_13213', 'COBRADE_13214',
       'COBRADE_13215', 'COBRADE_13322', 'COBRADE_14110', 'COBRADE_14120',
       'COBRADE_14131', 'COBRADE_14132', 'COBRADE_15110', 'COBRADE_15130',
       'COBRADE_15210', 'COBRADE_15230', 'COBRADE_22220', 'COBRADE_23120'

In [ ]:
# remove constant features (n0-nan)
constant_features = [
    feat for feat in X_train_no_nan.columns if X_train_no_nan[feat].std() == 0
]

print(constant_features)

X_train_no_nan.drop(labels=constant_features, axis=1, inplace=True)
X_test_no_nan.drop(labels=constant_features, axis=1, inplace=True)

X_train_no_nan.shape, X_test_no_nan.shape

#X_test.head()
print(X_test_no_nan.columns)

['COBRADE_11110', 'COBRADE_11120', 'COBRADE_11312', 'COBRADE_11313', 'COBRADE_13310', 'COBRADE_13321', 'COBRADE_14140', 'COBRADE_15120', 'COBRADE_22210', 'COBRADE_25100']
Index(['DH_total_danos_humanos', 'DM_total_danos_materiais',
       'PEPL_total_publico', 'PEPR_total_privado', 'regiao_Centro-oeste',
       'regiao_Nordeste', 'regiao_Norte', 'regiao_Sudeste', 'regiao_Sul',
       'COBRADE_11311', 'COBRADE_11321', 'COBRADE_11331', 'COBRADE_11332',
       'COBRADE_11340', 'COBRADE_11410', 'COBRADE_11420', 'COBRADE_11431',
       'COBRADE_11432', 'COBRADE_11433', 'COBRADE_12100', 'COBRADE_12200',
       'COBRADE_12300', 'COBRADE_13111', 'COBRADE_13112', 'COBRADE_13120',
       'COBRADE_13211', 'COBRADE_13212', 'COBRADE_13213', 'COBRADE_13214',
       'COBRADE_13215', 'COBRADE_13322', 'COBRADE_14110', 'COBRADE_14120',
       'COBRADE_14131', 'COBRADE_14132', 'COBRADE_15110', 'COBRADE_15130',
       'COBRADE_15210', 'COBRADE_15230', 'COBRADE_22220', 'COBRADE_23120',
       'COBRADE_2410

<ipython-input-11-edf55b6442ea>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_no_nan.drop(labels=constant_features, axis=1, inplace=True)
<ipython-input-11-edf55b6442ea>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_no_nan.drop(labels=constant_features, axis=1, inplace=True)


In [ ]:
# remove quasi-constant features
print(X_train.columns)
sel = VarianceThreshold(
    threshold=0.01)  # 0.1 indicates 99% of observations approximately


sel.fit(X_train)  # fit finds the features with low variance

print(sum(sel.get_support())) # how many not quasi-constant?

# COBRADE_11311', 'COBRADE_11321', 'COBRADE_11331', 'COBRADE_11332',
#        'COBRADE_11340', 'COBRADE_11410', 'COBRADE_11420', 'COBRADE_11431',
#        'COBRADE_11432', 'COBRADE_11433', COBRADE_13111', 'COBRADE_13112',  'COBRADE_13120',
      #  'COBRADE_13211', 'COBRADE_13212', COBRADE_13322', COBRADE_14132', 'COBRADE_15110', 'COBRADE_15130',
      # #  'COBRADE_15210', 'COBRADE_15230', 'COBRADE_22220', 'COBRADE_23120',
      # #  'COBRADE_24100', 'COBRADE_24200', 'COBRADE_25500',
      # 'COBRADE_11110', 'COBRADE_11120', 'COBRADE_11312', 'COBRADE_11313', 'COBRADE_13310', 'COBRADE_13321', 'COBRADE_14140', 'COBRADE_15120', 'COBRADE_22210', 'COBRADE_25100'


Index(['DH_total_danos_humanos', 'DM_total_danos_materiais',
       'PEPL_total_publico', 'PEPR_total_privado', 'regiao_Centro-oeste',
       'regiao_Nordeste', 'regiao_Norte', 'regiao_Sudeste', 'regiao_Sul',
       'COBRADE_11311', 'COBRADE_11321', 'COBRADE_11331', 'COBRADE_11332',
       'COBRADE_11340', 'COBRADE_11410', 'COBRADE_11420', 'COBRADE_11431',
       'COBRADE_11432', 'COBRADE_11433', 'COBRADE_12100', 'COBRADE_12200',
       'COBRADE_12300', 'COBRADE_13111', 'COBRADE_13112', 'COBRADE_13120',
       'COBRADE_13211', 'COBRADE_13212', 'COBRADE_13213', 'COBRADE_13214',
       'COBRADE_13215', 'COBRADE_13322', 'COBRADE_14110', 'COBRADE_14120',
       'COBRADE_14131', 'COBRADE_14132', 'COBRADE_15110', 'COBRADE_15130',
       'COBRADE_15210', 'COBRADE_15230', 'COBRADE_22220', 'COBRADE_23120',
       'COBRADE_24100', 'COBRADE_24200', 'COBRADE_25500',
       'DOMICILIO_AREARURAL', 'PDEFAGUA', 'PDEFESGOTO', 'PDEFLIXO', 'PDEFSAN',
       'QTDE_FAMILIAS_ATUALIZADAS'],
      dtype='obje

'DH_total_danos_humanos', 'DM_total_danos_materiais',
       'PEPL_total_publico', 'PEPR_total_privado', 'regiao_Centro-oeste',
       'regiao_Nordeste', 'regiao_Norte', 'regiao_Sudeste', 'regiao_Sul',
       'COBRADE_12100', 'COBRADE_12200', 'COBRADE_12300', 'COBRADE_13213',
       'COBRADE_13214', 'COBRADE_13215', 'COBRADE_14110', 'COBRADE_14120',
       'COBRADE_14131', 'DOMICILIO_AREARURAL', 'PDEFAGUA', 'PDEFESGOTO',
       'PDEFLIXO', 'PDEFSAN', 'QTDE_FAMILIAS_ATUALIZADAS'],
      dtype='object')

In [ ]:
# remove quasi-constant features (no nan)
sel_no_nan = VarianceThreshold(
    threshold=0.01)  # 0.1 indicates 99% of observations approximately

sel_no_nan.fit(X_train_no_nan)  # fit finds the features with low variance

sum(sel_no_nan.get_support()) # how many not quasi-constant?

24

In [ ]:
features_to_keep = X_train.columns[sel.get_support()]

In [ ]:
features_to_keep_n = X_train_no_nan.columns[sel_no_nan.get_support()]
print(features_to_keep, '\n', features_to_keep_n)

Index(['DH_total_danos_humanos', 'DM_total_danos_materiais',
       'PEPL_total_publico', 'PEPR_total_privado', 'regiao_Centro-oeste',
       'regiao_Nordeste', 'regiao_Norte', 'regiao_Sudeste', 'regiao_Sul',
       'COBRADE_12100', 'COBRADE_12200', 'COBRADE_12300', 'COBRADE_13213',
       'COBRADE_13214', 'COBRADE_13215', 'COBRADE_14110', 'COBRADE_14120',
       'COBRADE_14131', 'DOMICILIO_AREARURAL', 'PDEFAGUA', 'PDEFESGOTO',
       'PDEFLIXO', 'PDEFSAN', 'QTDE_FAMILIAS_ATUALIZADAS'],
      dtype='object') 
 Index(['DH_total_danos_humanos', 'DM_total_danos_materiais',
       'PEPL_total_publico', 'PEPR_total_privado', 'regiao_Centro-oeste',
       'regiao_Nordeste', 'regiao_Norte', 'regiao_Sudeste', 'regiao_Sul',
       'COBRADE_12100', 'COBRADE_12200', 'COBRADE_12300', 'COBRADE_13213',
       'COBRADE_13214', 'COBRADE_13215', 'COBRADE_14110', 'COBRADE_14120',
       'COBRADE_14131', 'DOMICILIO_AREARURAL', 'PDEFAGUA', 'PDEFESGOTO',
       'PDEFLIXO', 'PDEFSAN', 'QTDE_FAMILIAS_ATUALIZ

In [ ]:
# we can then remove the features like this
X_train = sel_no_nan.transform(X_train)
X_test = sel_no_nan.transform(X_test)

X_train.shape, X_test.shape

((9247, 24), (3964, 24))

In [ ]:
# we can then remove the features like this
X_train_no_nan = sel.transform(X_train_no_nan)
X_test_no_nan = sel.transform(X_test_no_nan)

X_train_no_nan.shape, X_test_no_nan.shape

((9213, 24), (3957, 24))

In [ ]:
print(type(X_train))
print(type(X_train_no_nan))
print(type(X_test))
print(type(X_test_no_nan))
print(type(y_train))
print(type(y_train_no_nan))
print(type(y_test))
print(type(y_test_no_nan))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [ ]:
# sklearn transformations lead to numpy arrays
# here I transform the arrays back to dataframes
# please be mindful of getting the columns assigned
# correctly

X_train= pd.DataFrame(X_train)
X_train.columns = features_to_keep

X_test= pd.DataFrame(X_test)
X_test.columns = features_to_keep

In [ ]:
# sklearn transformations lead to numpy arrays
# here I transform the arrays back to dataframes
# please be mindful of getting the columns assigned
# correctly (no nan)

X_train_no_nan= pd.DataFrame(X_train_no_nan)
X_train_no_nan.columns = features_to_keep_n

X_test_no_nan= pd.DataFrame(X_test_no_nan)
X_test_no_nan.columns = features_to_keep_n

In [ ]:
# check for duplicated features in the training set
duplicated_feat = []
for i in range(0, len(X_train.columns)):
    if i % 10 == 0:  # this helps me understand how the loop is going
        print(i)

    col_1 = X_train.columns[i]

    for col_2 in X_train.columns[i + 1:]:
        if X_train[col_1].equals(X_train[col_2]):
            duplicated_feat.append(col_2)

len(duplicated_feat)

0
10
20


0

In [ ]:
# check for duplicated features in the training set (no nan)
duplicated_feat_no_nan = []
for i in range(0, len(X_train_no_nan.columns)):
    if i % 10 == 0:  # this helps me understand how the loop is going
        print(i)

    col_1 = X_train_no_nan.columns[i]

    for col_2 in X_train_no_nan.columns[i + 1:]:
        if X_train_no_nan[col_1].equals(X_train[col_2]):
            duplicated_feat_no_nan.append(col_2)

len(duplicated_feat_no_nan)

0
10
20


0

In [ ]:
# remove duplicated features
X_train.drop(labels=duplicated_feat, axis=1, inplace=True)
X_test.drop(labels=duplicated_feat, axis=1, inplace=True)

X_train.shape, X_test.shape

((9247, 24), (3964, 24))

In [ ]:
X_train_no_nan.drop(labels=duplicated_feat, axis=1, inplace=True)
X_test_no_nan.drop(labels=duplicated_feat, axis=1, inplace=True)

X_train_no_nan.shape, X_test_no_nan.shape

((9213, 24), (3957, 24))

In [ ]:
# I keep a copy of the dataset except constant and duplicated variables
# to measure the performance of machine learning models
# at the end of the notebook

X_train_basic_filter = X_train.copy()
X_test_basic_filter = X_test.copy()

In [ ]:
# I keep a copy of the dataset except constant and duplicated variables
# to measure the performance of machine learning models
# at the end of the notebook

X_train_no_nan_basic_filter = X_train_no_nan.copy()
X_test_no_nan_basic_filter = X_test_no_nan.copy()

In [ ]:
# find and remove correlated features

def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns

    corr_matrix = dataset.corr()

    for i in range(len(corr_matrix.columns)):

        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

corr_features = correlation(X_train, 0.7)
corr_features_no_nan = correlation(X_train_no_nan, 0.7)
print('correlated features: ', len(set(corr_features)), corr_features )
print('correlated features (no nan): ', len(set(corr_features_no_nan)), corr_features_no_nan )

correlated features:  2 {'PDEFLIXO', 'PDEFSAN'}
correlated features (no nan):  2 {'PDEFLIXO', 'PDEFSAN'}


In [ ]:
X_train.drop(labels=corr_features, axis=1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)

X_train.shape, X_test.shape

((9247, 22), (3964, 22))

In [ ]:
X_train_no_nan.drop(labels=corr_features, axis=1, inplace=True)
X_test_no_nan.drop(labels=corr_features, axis=1, inplace=True)

X_train_no_nan.shape, X_test_no_nan.shape

((9213, 22), (3957, 22))

In [ ]:
# create a function to build random forests and compare performance in train and test set
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

def run_randomForests(X_train, X_test, y_train, y_test):
    rf = RandomForestClassifier(n_estimators=200, random_state=39, max_depth=4)
    rf.fit(X_train, y_train)
    print('Train set')
    pred = rf.predict_proba(X_train)
    print('Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
    print('Test set')
    pred = rf.predict_proba(X_test)
    print('Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

#     best_score = 0
#     best_tree_index = 0

#     for i, tree in enumerate(rf.estimators_):
#     # Previsão da árvore individual
#       y_pred_tree = tree.predict(X_test)

#     # Calcular o accuracy score da árvore nos dados de teste
#       score = accuracy_score(y_test, np.ravel(y_pred_tree))

#     # Encontrar a árvore com o melhor score
#       if score > best_score:
#           best_score = score
#           best_tree_index = i
#     print(f'A árvore com o melhor desempenho é a árvore {best_tree_index} com um score de {best_score:.4f}\n')

# # # Acessando a árvore com o melhor desempenho
#     best_tree = rf.estimators_[best_tree_index]

# # Opcional: Visualizar a árvore
#     plt.figure(figsize=(20, 15))
#     plot_tree(rf, filled=True, fontsize=10, feature_names=X.columns)
#     plt.show()




In [ ]:
# original dataset (all variables)
run_randomForests(X_train_original,
                  X_test_original,
                  y_train, y_test)

Train set
Random Forests roc-auc: 0.806602600300164
Test set
Random Forests roc-auc: 0.774244647147626


In [ ]:
# filter methods - basic
run_randomForests(X_train_basic_filter,
                  X_test_basic_filter,
                  y_train, y_test)


Train set
Random Forests roc-auc: 0.8149491455650816
Test set
Random Forests roc-auc: 0.7776588086277441


In [ ]:
# filter methods - correlation
run_randomForests(X_train,
                  X_test,
                  y_train, y_test)

Train set
Random Forests roc-auc: 0.81341569637481
Test set
Random Forests roc-auc: 0.7758063070862076


In [ ]:
# create a function to build logistic regression and compare performance in train and test set

def run_logistic(X_train, X_test, y_train, y_test):
    # function to train and test the performance of logistic regression
    logit = LogisticRegression(random_state=44, max_iter=500)
    logit.fit(X_train, y_train)
    print('Train set')
    pred = logit.predict_proba(X_train)
    print('Logistic Regression roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
    print('Test set')
    pred = logit.predict_proba(X_test)
    print('Logistic Regression roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

In [ ]:
# original
# for logistic regression features need to be in the same scale

# original
scaler = StandardScaler().fit(X_train_no_nan_original)

run_logistic(scaler.transform(X_train_no_nan_original),
             scaler.transform(X_test_no_nan_original), y_train_no_nan_original, y_test_no_nan_original)

Train set
Logistic Regression roc-auc: 0.7299574160481984
Test set
Logistic Regression roc-auc: 0.7296610428044055


In [ ]:
# filter methods - basic
scaler = StandardScaler().fit(X_train_no_nan_basic_filter)

run_logistic(scaler.transform(X_train_no_nan_basic_filter),
             scaler.transform(X_test_no_nan_basic_filter),
                  y_train_no_nan, y_test_no_nan)

Train set
Logistic Regression roc-auc: 0.7234023167384853
Test set
Logistic Regression roc-auc: 0.7342412362327241


In [ ]:
# filter methods - correlation
scaler = StandardScaler().fit(X_train_no_nan)

run_logistic(scaler.transform(X_train_no_nan),
             scaler.transform(X_test_no_nan),
                  y_train_no_nan, y_test_no_nan)

Train set
Logistic Regression roc-auc: 0.721493844304521
Test set
Logistic Regression roc-auc: 0.7313550660524148


In [ ]:
# Luciana Angrisano 15:06
# # Exemplo para o uso do StratifiedShuffleSplit.

# # Importação do módulo do StratifiedShuffleSplit.
# from sklearn.model_selection import StratifiedShuffleSplit

# # Usando o constructor para criar o objeto sss
# sss = StratifiedShuffleSplit(n_splits = 20,       # 20 simulações.
#                              test_size = 0.2,     # 20% do dataset será de testes.
#                              random_state = 42)   # Permitir a reprodutibilidade.

# # Criando um objeto do GridSearchCV
# grid_3 = GridSearchCV(estimator = clf,
#                       param_grid = parametros,
#                       cv = sss,
#                       scoring = 'f1')

# # Imprime o f1
# grid_3.fit(features,labels).best_score_
# Por fim, há a opção de não usar o cv do GridSearchCV, isto seria feito omitindo-o.
# # Exemplo do GridSearchCV sem cv. Demanda um cv manual.

# # Criando um objeto do GridSearchCV sem cv.
# grid_4 = GridSearchCV(estimator = clf,
#                       param_grid = parametros,
#                       scoring = 'f1')

# Bruna  to  Everyone 15:09
# https://neptune.ai/blog/hyperparameter-tuning-in-python-complete-guide
# https://domino.ai/blog/hyperopt-bayesian-hyperparameter-optimization

# Luciana Angrisano 15:12
# from skopt import BayesSearchCV
# from sklearn.ensemble import RandomForestClassifier

# # Definição do modelo
# model = RandomForestClassifier()

# # Definição do espaço de busca dos hiperparâmetros
# search_space = {
#     'n_estimators': (10, 500),  # Faixa de valores inteiros
#     'max_depth': (1, 50),        # Faixa de valores inteiros
#     'min_samples_split': (2, 10),
#     'min_samples_leaf': (1, 5),
#     'bootstrap': [True, False]   # Valores discretos
# }

# # Configuração do BayesSearchCV
# opt = BayesSearchCV(
#     estimator=model,
#     search_spaces=search_space,
#     n_iter=32,  # Número de iterações
#     cv=5,       # Validação cruzada com 5 dobras
#     n_jobs=-1,  # Paralelização
#     random_state=42
# )

# # Ajuste do modelo e busca dos hiperparâmetros ótimos
# opt.fit(X_train, y_train)

# # Hiperparâmetros ótimos
# print(opt.best_params_)